# 1.Import Data

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
data = np.abs(round(pd.read_csv('Data/Raw/data_2.csv'),6))
data

,prev_displ,prev_vell,prev_accl,prev_wspd,displ,vell,accl,wspd,act,dispm,velm,accm,dispm_0,velm_0,accm_0
0,0.000000,0.000000,0.00000,0.00,0.000101,0.000005,0.000060,2.860,10.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
1,0.000000,0.000000,0.00000,0.00,0.000100,0.000018,0.000051,2.860,20.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
2,0.000000,0.000000,0.00000,0.00,0.000099,0.000025,0.000035,2.860,30.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
3,0.000000,0.000000,0.00000,0.00,0.000098,0.000026,0.000017,2.860,40.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
4,0.000000,0.000000,0.00000,0.00,0.000096,0.000022,0.000006,2.860,50.0,0.000107,0.000146,0.000336,0.000108,0.000142,0.000336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23835,0.000077,0.000241,0.00019,10.34,0.000037,0.000175,0.000093,9.636,160.0,0.000124,0.000291,0.001836,0.000364,0.001143,0.003881
23836,0.000077,0.000241,0.00019,10.34,0.000036,0.000162,0.000076,9.636,170.0,0.000123,0.000265,0.001998,0.000364,0.001143,0.003881
23837,0.000077,0.000241,0.00019,10.34,0.000035,0.000152,0.000064,9.636,180.0,0.000122,0.000259,0.002159,0.000364,0.001143,0.003881
23838,0.000077,0.000241,0.00019,10.34,0.000063,0.000175,0.000094,9.636,190.0,0.000153,0.000352,0.002319,0.000364,0.001143,0.003881


In [3]:
#reward function = Ucondi/Uuncondi if 
for j in ['disp','vel','acc']:
    data['r_'+j] = np.abs(data[j+'m']/data[j+'m_0'])
    # if no change reward = 0
    data.loc[data['r_'+j]==1,'r_'+j] = 0 - data['act']/2000
    data.loc[data['r_'+j]>1,'r_'+j] = -data.loc[data['r_'+j]>1,'r_'+j] - data['act']/2000
    data.loc[(data['r_'+j]<1)&(data['r_'+j]>0),'r_'+j] = 1/data.loc[data['r_'+j]<1,'r_'+j] - data['act']/2000
    
    

# 2. Create Subset data

## 2.1 Check Data summary for rewards

In [5]:
for j in ['disp','vel','acc']:
    print(data['r_'+j].mean())
    print(data['r_'+j].median())
    print()

0.8060500215200775
1.1409633027522936

1.2620703814849
1.466666666666667

0.22583906399485526
0.9459776406805982



# 2.2 Separate the Data for models

In [6]:
for j in ['disp','vel','acc']:
    col_names = ['prev_'+j+'l', 'prev_wspd', j+'l', 'wspd', 'act', 'r_'+j]
    globals()[j] = round(data[col_names],6)
    eval(j)['state'] = list(zip( np.abs(eval(j)['prev_'+j+'l']),  eval(j)['prev_wspd']))
    eval(j)['state_p'] = list(zip( np.abs(eval(j)[j+'l']),  eval(j)['wspd']))
    globals()[j] = eval(j)[['state', 'state_p', 'act', 'r_'+j]].drop_duplicates()
    eval(j).to_csv('Data/{}.csv'.format(j))
# we have three datasets: disp, vel, acc, with s, s', a r
disp

,state,state_p,act,r_disp
0,"(0.0, 0.0)","(0.000101, 2.86)",10.0,1.004346
1,"(0.0, 0.0)","(0.0001, 2.86)",20.0,0.999346
2,"(0.0, 0.0)","(9.9e-05, 2.86)",30.0,0.994346
3,"(0.0, 0.0)","(9.8e-05, 2.86)",40.0,0.989346
4,"(0.0, 0.0)","(9.6e-05, 2.86)",50.0,0.984346
...,...,...,...,...
23835,"(7.7e-05, 10.34)","(3.7e-05, 9.636)",160.0,2.855484
23836,"(7.7e-05, 10.34)","(3.6e-05, 9.636)",170.0,2.874350
23837,"(7.7e-05, 10.34)","(3.5e-05, 9.636)",180.0,2.893607
23838,"(7.7e-05, 10.34)","(6.3e-05, 9.636)",190.0,2.284085


In [7]:
#check the total states vs observed next states
for j in [disp,vel,acc]:
    print('total samples:',len(j))
    print('total unique current state:', len(set(j['state'])))
    print('total unique next state:',len(set(j['state_p'])))
    print()

total samples: 22332
total unique current state: 900
total unique next state: 2667

total samples: 22687
total unique current state: 884
total unique next state: 2551

total samples: 22523
total unique current state: 887
total unique next state: 2791



In [8]:
for i in data.columns:
    print(i,':',len(set(data[i])))
# 20 actions
# 395 displacement
# 675 velocity
# 1026 acceleration

prev_displ : 185
prev_vell : 287
prev_accl : 382
prev_wspd : 154
displ : 217
vell : 432
accl : 887
wspd : 155
act : 20
dispm : 244
velm : 769
accm : 2438
dispm_0 : 252
velm_0 : 612
accm_0 : 905
r_disp : 18623
r_vel : 21833
r_acc : 18234


## 2.3 Unique Value of (parameter, wind_speed)

In [37]:
for i in ['displ','vell','accl']:
    temp = set(pd.concat([data['prev_'+i],data[i]], axis=0,ignore_index=True))
    temp2 = set(pd.concat([data['prev_wspd'],data['wspd']], axis=0,ignore_index=True))
    globals()['unique_{}'.format(i)] = [(j,k) for j in temp for k in temp2]
len(unique_displ),len(unique_vell),len(unique_accl)
(33635, 67270, 138725)

(33635, 67270, 138725)

# 4. Dictionarized all (disp/vel/acc, wind) to s1,s2...

In [38]:
#create dictionary for each state
for i in ['unique_displ','unique_vell','unique_accl']:
  my_list = eval(i)
  globals()['dict_'+i] = dict() 
  for index,value in enumerate(my_list):
    eval('dict_'+i)[index+1] = value
#dict_unique_prev_displ
len(dict_unique_displ.keys())

33635

## 4.1 Conver the tuple states into a dictionarized states 

In [39]:

for i in ['disp','vel','acc']:
    print(i)
    index = 0
    temp = eval(i)
    dic_i = eval('dict_unique_'+i+'l')
    key_list = list(dic_i.keys())
    val_list = list(dic_i.values())

    temp['s'] = 0
    temp['s_p'] = 0

    for z in range(len(temp)):
        j = temp['state'].iloc[z]
        k = temp['state_p'].iloc[z]
        position_s = val_list.index(j)
        position_s_p = val_list.index(k)
        temp['s'].iloc[index] = key_list[position_s]
        temp['s_p'].iloc[index] = key_list[position_s_p]
        index +=1
    temp['a'] = temp['act']/10
    temp['a']  = temp['a'].astype(int)
    globals()[i] = temp
#my_dict_unique_displ.get()

disp


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


vel
acc


In [40]:
vel['s'].min()

1

In [41]:
for i in ['disp','vel','acc']:
    globals()[i+'o'] = eval(i)[['s','a','r_'+i,'s_p']].rename(columns={ 'r_'+i: 'r','s_p' : 'sp'})

# 4.2 Save the Data to Data file
* csv
* Dictionary

In [42]:
for i in ['disp','vel','acc']:
    eval(i+'o').to_csv('Data/Version_from_python/{}.csv'.format(i+'o'),index=False)

    dic_i = eval('dict_unique_prev_'+i+'l')
    a_file = open("Data/{}_dic.json".format(i), "w")
    json.dump(dic_i, a_file)
    a_file.close()

### Loading Test

In [43]:
for i in ['disp','vel','acc']:
    a_file = open("Data/Version_from_python/{}_dic.json".format(i), "r")
    output = a_file.read()
    globals()['{}_dic'.format(i)]=output

In [44]:
set(dispo['a'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}

In [45]:
len(set(velo['sp']))

2551

In [115]:
len(velo)

17721

In [114]:
set(acco['a'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}

In [111]:
# ratio =  max(DO) / max(NOTHING)
    # if ratio = 1 -> reward = 0 + penalty
    # if ratio > 1 -> reward = -ratio + penalty
    # if ratio < 1 -> reward = 1/ratio + penalty

# reward = max(NOTHING) / max(DO) - Voltage / constant 

In [ ]:
velo